## Imports

In [1]:
import copy
import numpy as np
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from config import config
import os
import sys
current_dir = os.getcwd()
path = "C:\\Users\\eirik\\Documents\\Master\\ISLBBNN\\islbbnn"
# path = "C:\\you\\path\\to\\islbbnn\\folder\\here"
os.chdir(path)
import plot_functions as pf
import pipeline_functions as pip_func
import importlib
importlib.reload(pf)
importlib.reload(pip_func)
sys.path.append('networks')
from flow_sigmoid_net import BayesianNetwork

os.chdir(current_dir) # set the working directory back to this one 

CPUs are used!


# Attain data

Problem:

$$y = x_1 + x_2 + 100 +\epsilon$$

where $\epsilon \sim N(0,0.01)$. 


Can make $x_3$ dependent on $x_1$. The depedence is defined in the following way:

\begin{align*}
 x_1 &\sim Unif(-10,10) \\
 x_3 &\sim Unif(-10,10) \\
 x_3 &= \text{dep}\cdot x_1 + (1-\text{dep})\cdot x_3
\end{align*}

## Pre process and batch size

In [2]:
# define parameters
HIDDEN_LAYERS = config['n_layers'] - 2 
epochs = config['num_epochs']
post_train_epochs = config['post_train_epochs']
dim = config['hidden_dim']
num_transforms = config['num_transforms']
n_nets = config['n_nets']
n_samples = config['n_samples']
lr = config['lr']
class_problem = config["class_problem"]
non_lin = config["non_lin"]
verbose = config['verbose']
save_res = config['save_res']
patience = config['patience']
SAMPLES = 1


# Get linear data, here a regression problem
y, X = pip_func.create_data_unif(n_samples, beta=[100,1,1,1,1], dep_level=0.0, classification=class_problem)

n, p = X.shape  # need this to get p 
print(n,p,dim)

# Define BATCH sizes
BATCH_SIZE = int((n*0.8)/100)
TEST_BATCH_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_BATCH_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

TRAIN_SIZE = int((n*0.80)/100)
TEST_SIZE = int(n*0.10) # Would normally call this the "validation" part (will be used during training)
VAL_SIZE = int(n*0.10) # and this the "test" part (will be used after training)

NUM_BATCHES = TRAIN_SIZE/BATCH_SIZE

print(NUM_BATCHES)

assert (TRAIN_SIZE % BATCH_SIZE) == 0
assert (TEST_SIZE % TEST_BATCH_SIZE) == 0

40000 4 10
1.0


## Seperate a test set for later

In [3]:
# Split keep some of the data for validation after training
X, X_test, y, y_test = train_test_split(
    X, y, test_size=0.10, random_state=42)#, stratify=y)

test_dat = torch.tensor(np.column_stack((X_test,y_test)),dtype = torch.float32)

# Train network

## Device setup

In [4]:
# select the device and initiate model

# DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
LOADER_KWARGS = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}

## Train, validate, and test network

In [5]:
all_nets = {}
metrics_several_runs = []
metrics_median_several_runs = []
for ni in range(n_nets):
    post_train = False
    print('network', ni)
    # Initate network
    torch.manual_seed(ni+42)
    net = BayesianNetwork(dim, p, HIDDEN_LAYERS, classification=class_problem, num_transforms=num_transforms).to(DEVICE)
    alphas = pip_func.get_alphas_numpy(net)
    nr_weights = np.sum([np.prod(a.shape) for a in alphas])
    print(nr_weights)

    optimizer = optim.Adam(net.parameters(), lr=lr)
    
    all_nll = []
    all_loss = []

    # Split into training and test set
    X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=1/9, random_state=ni)#, stratify=y)
            
    train_dat = torch.tensor(np.column_stack((X_train,y_train)),dtype = torch.float32)
    val_dat = torch.tensor(np.column_stack((X_val,y_val)),dtype = torch.float32)
    
    # Train network
    counter = 0
    highest_acc = 0
    best_model = copy.deepcopy(net)
    for epoch in range(epochs + post_train_epochs):
        if verbose:
            print(epoch)
        nll, loss = pip_func.train(net, train_dat, optimizer, BATCH_SIZE, NUM_BATCHES, p, DEVICE, nr_weights, post_train=post_train)
        nll_val, loss_val, ensemble_val = pip_func.val(net, val_dat, DEVICE, verbose=verbose, reg=(not class_problem), post_train=post_train)
        if ensemble_val >= highest_acc:
            counter = 0
            highest_acc = ensemble_val
            best_model = copy.deepcopy(net)
        else:
            counter += 1
        
        all_nll.append(nll)
        all_loss.append(loss)

        if epoch == epochs-1:
            post_train = True   # Post-train --> use median model 
            for name, param in net.named_parameters():
                for i in range(HIDDEN_LAYERS+1):
                    #if f"linears{i}.lambdal" in name:
                    if f"linears.{i}.lambdal" in name:
                        param.requires_grad_(False)

        if counter >= patience:
            break
        
    all_nets[ni] = net 
    # Results
    metrics, metrics_median = pip_func.test_ensemble(all_nets[ni],test_dat,DEVICE,SAMPLES=10, reg=(not class_problem)) # Test same data 10 times to get average 
    metrics_several_runs.append(metrics)
    metrics_median_several_runs.append(metrics_median)
    pf.run_path_graph(all_nets[ni], threshold=0.5, save_path=f"path_graphs/flow/prob/test{ni}", show=verbose)

if verbose:
    print(metrics)
m = np.array(metrics_several_runs)
m_median = np.array(metrics_median_several_runs)

network 0
54
0
loss 564.9293823242188
nll 17.257888793945312
density 0.7314447390812414

val_loss: 707.6798, val_nll: 158.2198, val_ensemble: 0.9895, used_weights_median: 38

1
loss 166.2592010498047
nll 12.042278289794922
density 0.25178228139325426

val_loss: 284.4129, val_nll: 143.6056, val_ensemble: 0.9862, used_weights_median: 8

2
loss 79.34066772460938
nll 14.564138412475586
density 0.18172707091327067

val_loss: 264.0968, val_nll: 205.6084, val_ensemble: 0.9760, used_weights_median: 8

3
loss 28.61278533935547
nll 15.651078224182129
density 0.1655927726102096

val_loss: 382.7363, val_nll: 370.3861, val_ensemble: 0.9643, used_weights_median: 8

4
loss -0.7720432281494141
nll 15.476224899291992
density 0.13925996288243267

val_loss: 97.7263, val_nll: 112.5806, val_ensemble: 0.9900, used_weights_median: 5

5
loss -21.119325637817383
nll 27.09465980529785
density 0.1253015967430892

val_loss: 58.5322, val_nll: 117.6861, val_ensemble: 0.9868, used_weights_median: 2

6
loss -71.20240

In [6]:
pf.run_path_graph_weight(net, save_path="path_graphs/flow/weight/temp", show=True, flow=True)

In [7]:
pip_func.get_alphas_numpy(net)

[array([[0.00633232, 0.00682793, 0.0042301 , 0.00684217],
        [0.00585916, 0.00680713, 0.00414927, 0.00638634],
        [0.05251874, 0.18215206, 0.00663297, 0.00622487],
        [0.00767064, 0.00753264, 0.00379462, 0.00553425],
        [0.00664885, 0.00813247, 0.00496843, 0.00583838],
        [0.00610813, 0.00765855, 0.00389322, 0.00615199],
        [0.00552204, 0.00584669, 0.00163778, 0.00628702],
        [0.00673197, 0.00750938, 0.00369952, 0.00653431],
        [0.99996316, 0.9988244 , 0.00544006, 0.00563788],
        [0.00622068, 0.00844101, 0.00341081, 0.00590927]], dtype=float32),
 array([[4.1774323e-04, 5.5158575e-04, 6.9025980e-04, 2.2072923e-04,
         4.9270829e-04, 3.5440543e-04, 4.1501515e-04, 3.0016989e-04,
         1.4086608e-02, 7.3907671e-05, 9.9999464e-01, 9.9999988e-01,
         1.3635819e-03, 5.4842066e-03]], dtype=float32)]

In [8]:
pip_func.weight_matrices_numpy(net)

[array([[ 0.5592295 ,  0.44075483,  0.09219293,  0.85406816],
        [ 0.35003656,  0.4219904 , -0.08232986,  0.14748089],
        [-0.6021428 , -0.34066555,  0.27757537,  0.05496798],
        [ 0.90761817,  1.0492636 , -0.02676811, -0.21698482],
        [ 0.9758332 ,  0.8083283 , -0.04013633,  0.09636123],
        [ 0.56553346,  0.46006137,  0.13256216,  0.17106597],
        [ 0.48603112,  0.35061854,  0.19707102,  0.31749308],
        [ 0.6637901 ,  0.5491373 , -0.10293567,  0.11483766],
        [-0.00133723,  0.01921339,  0.01942176,  0.01242522],
        [ 0.43062106,  0.47798884, -0.00203214, -0.00222193]],
       dtype=float32),
 array([[ 4.6113085e-02, -3.1549659e-01,  1.8999761e-01,  2.5166939e-03,
          2.6167503e-03,  8.3691822e-03,  1.2796061e-01,  3.6246911e-01,
         -3.1050256e-01, -7.6871586e-04,  2.3277228e+00, -2.6576648e+00,
         -1.0083407e-02, -9.1972956e-03]], dtype=float32)]